In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
import torch.optim as optim
import time
import os
import copy
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,InputLayer, Conv2D, Conv2DTranspose, Flatten, Dropout, BatchNormalization, LeakyReLU, Input, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
#from tensorflow.keras.losses import CategoricalCrossentropy
#from tensorflow.keras.optimizers import Adam

In [ ]:
basePath = '/content/'

modelSave = basePath+'weights.pth'

data_dir = basePath+'Sample'
#data_dir = basePath+'Dataset_2'

batch_size = 2
num_epochs = 1
input_size = 224
NUM_CLASSES = 3

In [ ]:
image_types = ["Covid-19","No_findings","Pneumonia"]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def resize_images(image_dir):
  for im_type in image_types:
    # Iterate through each image file in each image_type folder
    #  glob reads in any image with the extension "image_dir/im_type/*"
    for file in glob.glob(os.path.join(image_dir, im_type, "*")):
        im = Image.open(file)
        f, e = os.path.splitext(file)
        imResize = im.resize((input_size,input_size), Image.ANTIALIAS)
        os.remove(file)
        imResize.save(f + '.png', 'PNG', quality=90)

In [ ]:
'''!cp -R '/content/drive/My Drive/COVID-19/Sample' '/content/'
#!cp -R '/content/drive/My Drive/COVID-19/Dataset_2' '/content/'
resize_images(data_dir+'/train')
resize_images(data_dir+'/val')'''

"!cp -R '/content/drive/My Drive/COVID-19/Sample' '/content/'\n#!cp -R '/content/drive/My Drive/COVID-19/Dataset_2' '/content/'\nresize_images(data_dir+'/train')\nresize_images(data_dir+'/val')"

In [ ]:
'''resize_images(data_dir+'/train')
resize_images(data_dir+'/val')'''

"resize_images(data_dir+'/train')\nresize_images(data_dir+'/val')"

In [ ]:
#!rm -R 'Sample'

In [ ]:
class DenseNet121(nn.Module):
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x
    

class CovidAID(DenseNet121):
    def __init__(self):
        #NUM_CLASSES = 3
        super(CovidAID, self).__init__(NUM_CLASSES)

In [ ]:
def conv_block(out_channels, filter_dim=3, stride=1):
    # for_pad = lambda s: s if s > 2 else 3
    block = Sequential()
    '''if filter_dim == 1:
      block.add(Conv2DTranspose(out_channels,3, strides=(1,1), padding='valid', activation=None))
    else:
      block.add(Conv2D(out_channels, filter_dim, strides=(1,1), padding='same', activation=None))'''
    block.add(Conv2D(out_channels, filter_dim, strides=(1,1), padding='same', activation=None))
    block.add(BatchNormalization())
    block.add(LeakyReLU(alpha=0.1))
    return block

In [ ]:
def triple_conv(out_channels, in_channels):
    block = Sequential()
    block.add(conv_block(out_channels, 3))

    block.add(conv_block(in_channels, filter_dim=1))
    '''block.add(Conv2DTranspose(in_channels,3, strides=(1,1), padding='valid', activation=None))
    block.add(BatchNormalization())
    block.add(LeakyReLU(alpha=0.1))'''

    block.add(conv_block(out_channels, 3))
    return block

In [ ]:
def darkCovidNet():
    model = Sequential()
    model.add(Input(shape=(input_size,input_size, 3)))
    model.add(conv_block(8, 3))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(conv_block(16, 3))
    model.add(MaxPooling2D())
    model.add(triple_conv(32, 16))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(triple_conv(64, 32))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(triple_conv(128, 64))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(triple_conv(256, 128))
    model.add(conv_block(128, filter_dim=1))
    model.add(conv_block(256, 3))
    model.add(conv_block(3, 3))
    model.add(Flatten())
    #model.add(Dense(192))
    model.add(Dense(3))
    return model

In [ ]:
def PKNet():
    model = Sequential()
    model.add(InputLayer(input_shape=(input_size , input_size , 3)))

    # 1st conv block

    model.add(Conv2D(8, (3, 3), activation='relu', strides=(1, 1), padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization())

    # 2nd conv block

    model.add(Conv2D(16, (3, 3), activation='relu', strides=(1, 1), padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization())

    # 3rd conv block

    model.add(Conv2D(32, (3, 3), activation='relu', strides=(1, 1), padding='same'))
    model.add(BatchNormalization())

    # 4th conv block

    model.add(Conv2D(16, (3, 3), activation='relu', strides=(1,1), padding='same'))
    model.add(BatchNormalization())

    # 5th conv block

    model.add(Conv2D(32, (3, 3), activation='relu', strides=(1,1), padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization())

    # 6th conv block

    model.add(Conv2D(64, (3, 3), activation='relu', strides=(1,1), padding='same'))
    model.add(BatchNormalization())

    # 7th conv block

    model.add(Conv2D(32, (3, 3), activation='relu', strides=(1,1), padding='same'))
    model.add(BatchNormalization())

    # 8th conv block

    model.add(Conv2D(64, (3, 3), activation='relu', strides=(1,1), padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
    model.add(BatchNormalization())

    # 9th conv block

    model.add(Conv2D(128, (3, 3), activation='relu', strides=(1,1), padding='same'))
    model.add(BatchNormalization())

    # 10th conv bock

    model.add(Conv2D(64, (3, 3), activation='relu', strides=(1,1), padding='same'))
    model.add(BatchNormalization())

    # ANN block

    model.add(Flatten())
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dropout(0.25))

    # output layer

    model.add(Dense(units=3, activation='softmax'))
    return model

In [ ]:
def getTrainDataLoaders():
    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

    print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
    # Create training and validation dataloaders
    idx2label_dict = {v: k for k, v in image_datasets['train'].class_to_idx.items()}
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

    return idx2label_dict,dataloaders_dict

In [ ]:
def saveInFile(features,labels,file_name):
    saveData = np.concatenate((features,np.asarray([labels]).T),axis=1)
    np.savetxt(file_name,saveData, delimiter=',',fmt='%8.6f')

In [ ]:
def train_model(model,tfmodel,tfmodel2, dataloaders, soft_mx, num_epochs=25):

    since = time.time()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        #for phase in ['val']:
        for phase in ['train', 'val']:
            model.eval()

            #-------1-------
            y_true = np.asarray([],dtype=np.int32)
            y_pred_aid = np.asarray([],dtype=np.int32)
            #-------2-------
            y_pred_dark = np.asarray([],dtype=np.int32)
            #-------3-------
            y_pred_pknet = np.asarray([],dtype=np.int32)
            #---------------
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                #-------1-------
                outputs = model(inputs)
                outputs = soft_mx(outputs)
                outputs = outputs.detach().cpu().numpy()
                if y_pred_aid.shape[0] == 0:
                    y_pred_aid =  outputs
                else:
                    y_pred_aid = np.append(y_pred_aid, outputs, axis=0)
                #--------2------
                tfinputs = inputs.data.detach().cpu().numpy()
                tfinputs = np.transpose(tfinputs,(0,2,3,1))
                outputs = tfmodel(tfinputs)
                outputs = tf.keras.activations.softmax(outputs).numpy()
                #print("1tf:",outputs.shape)
                if y_pred_dark.shape[0] == 0:
                    y_pred_dark =  outputs
                else:
                    y_pred_dark = np.append(y_pred_dark, outputs, axis=0)
                #--------3------
                outputs = tfmodel2(tfinputs)
                outputs = tf.keras.activations.softmax(outputs).numpy()
                #print("2tf:",outputs.shape)
                if y_pred_pknet.shape[0] == 0:
                    y_pred_pknet =  outputs
                else:
                    y_pred_pknet = np.append(y_pred_pknet, outputs, axis=0)
                #---------------

                y_true = np.append(y_true,labels.data.detach().cpu().numpy())
                #break

            saveInFile(y_pred_aid,y_true,'dense_out_'+phase+'.csv')
            #print("1shape:",y_pred_dark.shape,y_true.shape)
            saveInFile(y_pred_dark,y_true,'dark_out_'+phase+'.csv')
            #print("2shape:",y_pred_pknet.shape,y_true.shape)
            saveInFile(y_pred_pknet,y_true,'pknet_out_'+phase+'.csv')
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    #saveInFile(y_pred,y_true,'dense_out1.csv')

In [ ]:
model1 = CovidAID()
model1.load_state_dict( torch.load(modelSave,map_location=torch.device('cpu')))
model1 = model1.to(device)

In [ ]:
model2 = darkCovidNet()
model2.load_weights('CovidModel.h5')

In [ ]:
model3 = PKNet()
model3.load_weights('PKModel.h5')

In [ ]:
#print(model)

In [ ]:
idx2label_dict,dataloaders = getTrainDataLoaders()

Initializing Datasets and Dataloaders...


In [ ]:
# Observe that all parameters are being optimized
soft_mx = nn.Softmax(dim=1)
train_model(model1,model2,model3, dataloaders, soft_mx, num_epochs=num_epochs)

Epoch 0/0
----------

Training complete in 0m 4s
